In [ ]:
import logging

logging.basicConfig(level=logging.DEBUG)

In [3]:
import yfinance as yf

ticker = "AAPL"
prices = yf.Ticker(ticker).history(period="1y")

prices['high'] = prices['High']
prices['low'] = prices['Low']
prices['close'] = prices['Close']
prices['open'] = prices['Open']

In [3]:
from mplchart.zigzag import Zigzag

zigzag = Zigzag(backcandels=2, forwardcandels=2, pivot_limit=100, offset=0)
zigzag.calculate(prices)

for pivot in zigzag.zigzag_pivots:
    print(f"Pivot: time={pivot.point.time}, index={pivot.point.index}, price={pivot.point.price}, direction={pivot.direction}")


Pivot: time=2024-11-14 00:00:00-05:00, index=250, price=228.8699951171875, direction=2
Pivot: time=2024-11-11 00:00:00-05:00, index=247, price=221.5, direction=-1
Pivot: time=2024-11-08 00:00:00-05:00, index=246, price=228.66000366210938, direction=1
Pivot: time=2024-11-04 00:00:00-05:00, index=242, price=219.46854635353304, direction=-2
Pivot: time=2024-10-28 00:00:00-04:00, index=237, price=234.472033847566, direction=1
Pivot: time=2024-10-23 00:00:00-04:00, index=234, price=227.50969037853835, direction=-1
Pivot: time=2024-10-15 00:00:00-04:00, index=228, price=237.22900130875718, direction=2
Pivot: time=2024-10-07 00:00:00-04:00, index=222, price=221.08676038639473, direction=-2
Pivot: time=2024-10-04 00:00:00-04:00, index=221, price=227.74942888869526, direction=1
Pivot: time=2024-10-02 00:00:00-04:00, index=219, price=222.77491454853467, direction=-2
Pivot: time=2024-09-30 00:00:00-04:00, index=217, price=232.7439422607422, direction=1
Pivot: time=2024-09-25 00:00:00-04:00, index

In [3]:
from mplchart.trendline_patterns import find_trend_lines, TrendLinePattern, TrendLineProperties
from typing import List

scan_props = TrendLineProperties(offset=0, number_of_pivots=5, error_ratio=0.02)

# Initialize pattern storage
patterns: List[TrendLinePattern] = []

# Find patterns
for i in range(0, len(zigzag.zigzag_pivots)):
    find_trend_lines(zigzag, i, scan_props, patterns, prices)

for pattern in patterns:
    print(f"{pattern.pattern_name}: {pattern.pivots[0].point.index} - {pattern.pivots[-1].point.index}")


Ranging Channel: 222 - 242
Ascending Channel: 178 - 202
Descending Triangle (Expanding): 171 - 193
Falling Wedge (Expanding): 163 - 191
Rising Wedge (Expanding): 149 - 171
Rising Wedge (Contracting): 142 - 163
Rising Wedge (Contracting): 140 - 161
Rising Wedge (Expanding): 131 - 160
Rising Wedge (Expanding): 129 - 149
Falling Wedge (Expanding): 55 - 77
Descending Channel: 52 - 75
Converging Triangle: 45 - 66
Falling Wedge (Contracting): 40 - 63


In [3]:
from mplchart.reversal_patterns import find_reversal_patterns, ReversalPatternProperties

scan_props = ReversalPatternProperties(min_periods_lapsed=8, flat_ratio=0.1)
patterns = []
for i in range(0, len(zigzag.zigzag_pivots)):
    find_reversal_patterns(zigzag, i, scan_props, patterns, prices)

for pattern in patterns:
    print(f"{pattern.pattern_name}: {pattern.pivots[0].point.index} - {pattern.pivots[-1].point.index}")

In [2]:
from mplchart.rsi_div_patterns import RsiDivergenceProperties, find_rsi_divergences

patterns = []
scan_props = RsiDivergenceProperties(min_periods_lapsed=5, flat_ratio=0.01)
find_rsi_divergences(2, 2, scan_props, patterns, prices)

for pattern in patterns:
    print(f"{pattern.pattern_name}: {pattern.divergence_line.p1.index} - {pattern.divergence_line.p1.price}")


Hidden Bearish: 66.0 - 41.951761639866454
Bearish: 125.0 - 74.12529920316314
Bearish: 158.0 - 78.76851895755524
Bullish: 77.0 - 32.132291954746364
Bullish: 86.0 - 37.3682563815063
Hidden Bullish: 118.0 - 64.13655142790529


In [4]:
from mplchart.rsi_div_patterns import calc_rsi
from mplchart.zigzag import normalize_price, window_peaks

prices = normalize_price(prices)
rsi = calc_rsi(prices)
rsi_highs, rsi_lows = window_peaks(rsi, 2, 2)


In [5]:
rsi_high_pivots = rsi.where(rsi == rsi_highs)
rsi_low_pivots = rsi.where(rsi == rsi_lows)

In [6]:
import pandas as pd
# Create DataFrame with row numbers
prices_with_row = prices.copy()
prices_with_row['row_number'] = pd.RangeIndex(len(prices))

# Merge for highs - including RSI values
rsi_pivots= pd.merge(
    pd.DataFrame({'rsi_high': rsi_high_pivots, 'rsi_low': rsi_low_pivots}),  # Convert Series to DataFrame with column name
    prices_with_row[['row_number', 'norm_high', 'norm_low']],
    left_index=True,
    right_index=True,
    how='left'
)

In [7]:
rsi_pivots[['rsi_high', 'norm_high','row_number']].dropna()


,rsi_high,norm_high,row_number
Date,,,
2023-12-14 00:00:00-05:00,68.225090,0.571734,17
2023-12-28 00:00:00-05:00,49.707363,0.516947,26
2024-01-10 00:00:00-05:00,40.624659,0.425707,34
2024-01-23 00:00:00-05:00,62.181057,0.528987,42
2024-02-07 00:00:00-05:00,50.376569,0.477071,53
2024-02-22 00:00:00-05:00,44.097861,0.412408,63
2024-02-27 00:00:00-05:00,41.951794,0.400905,66
2024-03-20 00:00:00-04:00,52.494660,0.342840,82
2024-03-27 00:00:00-04:00,45.392383,0.286765,87
